# Deep Dive Spark SQL & Window Functions

Ce notebook est destiné à vous faire passer au niveau supérieur. Les **Window Functions** sont l'outil le plus puissant pour l'analyse de séries temporelles et de classements en SQL/Spark.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, rank, lag, lead, desc, sum as _sum

spark = SparkSession.builder.appName("Spark SQL Deep Dive").master("local[*]").getOrCreate()

## 1. Window Functions (Fonctions de Fenêtrage)

Imaginez que vous voulez comparer le salaire d'un employé avec la moyenne de SON département, ou voir l'évolution des ventes d'un jour à l'autre.

On définit une "Fenêtre" (Window) qui partitionne les données.

In [ ]:
# Données : Ventes par Vendeur et par Jour
data_ventes = [
    ("2023-01-01", "Alice", 100),
    ("2023-01-02", "Alice", 150),
    ("2023-01-03", "Alice", 120),
    ("2023-01-01", "Bob", 200),
    ("2023-01-02", "Bob", 180),
    ("2023-01-03", "Bob", 250)
]
df_ventes = spark.createDataFrame(data_ventes, ["date", "vendeur", "montant"])

# Objectif : Calculer la différence de vente par rapport à la veille pour CHAQUE vendeur

In [ ]:
# 1. Définir la Fenêtre : On partitionne par Vendeur et on trie par Date
windowSpec = Window.partitionBy("vendeur").orderBy("date")

# 2. Utiliser LAG (valeur précédente)
df_lag = df_ventes.withColumn("montant_veille", lag("montant", 1).over(windowSpec))

# 3. Calculer la différence
df_diff = df_lag.withColumn("progression", col("montant") - col("montant_veille"))

df_diff.show()

## 2. Classements (Rank vs Dense Rank)

Établir un classement des meilleurs vendeurs.

In [ ]:
windowRank = Window.orderBy(desc("montant"))

df_rank = df_ventes.withColumn("classement_global", rank().over(windowRank))
df_rank.show()

## 3. Complexes Types (Arrays & Maps)

Spark gère très bien les données imbriquées (comme du JSON).

In [ ]:
from pyspark.sql.functions import explode, split

# Données : Un livre et une liste de tags (sous forme de chaîne)
data_books = [
    ("Harry Potter", "magie,jeunesse,fantasy"),
    ("Le Seigneur des Anneaux", "fantasy,aventure,epique")
]
df_books = spark.createDataFrame(data_books, ["titre", "tags_str"])

# Convertir la chaîne en Array
df_arrays = df_books.withColumn("tags_array", split(col("tags_str"), ","))

# EXPLODE : Créer une ligne par tag (très utile !)
df_exploded = df_arrays.select("titre", explode("tags_array").alias("tag_unique"))

df_exploded.show()

In [ ]:
spark.stop()